In [2]:
using DataStructures

In [ ]:
function la_fontaine_cycle(dish, f_handler)
    t = deepcopy(dish) # The tortoise
    h = deepcopy(dish) # The haze
    t = f_handler(t)
    h = f_handler(f_handler(h))
    nb_t = 1
    while t != h
        t = f_handler(t)
        h = f_handler(f_handler(h))
        nb_t += 1
    end
    # nb_t is divisible by the period of the cycle. 
    # Let's figure out when the cycle start 
    
    start_point = 0
    t = deepcopy(dish)
    while t != h
        t = f_handler(t)
        h = f_handler(h)
        start_point += 1
    end

    # Now, find the shortest cycle
    c = 1
    h = f_handler(h)
    while t != h
        c += 1
        h = f_handler(h)
    end
    return (c, start_point)
end

In [5]:
function handle_signal(from, to, is_high, modules)
    current_type = modules[to]["type"]
    # println("Current type is $current_type")

    if current_type == "%"
        # Flip - flop 
        if !is_high
            modules[to]["is_off"] = !modules[to]["is_off"]
            return !modules[to]["is_off"]
        end # Else, do nothing
    elseif current_type == "&"
        # Conjunction module
        # println(modules[to]["states"])
        modules[to]["states"][from] = is_high
        # println(modules[to]["states"])
        return !all([modules[to]["states"][k] for k in keys(modules[to]["states"])])
    elseif current_type == "broadcaster" # Shouldbe a broadcaster module
        return is_high
    end
    return nothing
end

handle_signal (generic function with 1 method)

In [ ]:
function one_button_push(states)
    nb_low = 0
    nb_high = 0

    # A signal has three components: from, to, is_high
    queuing = [("button", "broadcaster", false)]

    while length(queuing) > 0 
        (from, to, is_high) = popfirst!(queuing)
        if is_high
            nb_high += 1
        else
            nb_low += 1
        end
        # println((from, to, is_high))
        next_bool = handle_signal(from, to, is_high, states)
        if next_bool != nothing
            for n in states[to]["output"]
                push!(queuing, (to, n, next_bool))
            end
        end
        # println("Queuing is $queuing")
        # println(states)
    end
    # println(states)
    
    return nb_high, nb_low
end

In [ ]:
(p) = open("20_1.txt") do f 
    nb_high = 0
    nb_low = 0

    all_modules = DefaultDict(() -> DefaultDict(() -> Dict()))
    all_output_keys = Set()
    # all_modules["output"] = Dict("type" => "output", "input" => [], "states" => Dict(), "output" => [], "is_off" => true)
    
    for l in eachline(f)
        (emitter, receiver) = split(l, " -> ")
        mod_name = emitter[2:end]
        type_name = emitter[1:1]
        if emitter == "broadcaster"
            mod_name = emitter
            type_name = emitter
        end
        for o in split(receiver, ", ")
            push!(all_output_keys, o)
        end
        
        all_modules[mod_name] = Dict("type" => type_name, "input" => [], "states" => Dict(), "output" => split(receiver, ", "), "is_off" => true)
            
    end

    for o in all_output_keys
        if !(o in keys(all_modules ))
            all_modules[o] = Dict("type" => "void", "input" => [], "states" => Dict(), "output" => [], "is_off" => true)
        end
    end
    
    
    println(all_modules)

    for mod_name in keys(all_modules)
        for a_receiver in all_modules[mod_name]["output"]
            println("Current mod name is $mod_name and receiver is $a_receiver")
            push!(all_modules[a_receiver]["input"], mod_name)
            all_modules[a_receiver]["states"][mod_name] = false # false means low pulse in memory
        end
    end

    println(all_modules)

    for i in range(1,1000)
        high_c, low_c = one_button_push(all_modules)
        nb_high += high_c
        nb_low += low_c
    end

    println("nb high is $nb_high qnd nb low is $nb_low")
    
    (nb_high * nb_low)
end

In [ ]:
function empty_emptiness(a,b)
    c = a+b
    return nothing
end

In [ ]:
empty_emptiness(1,2) == nothing

In [ ]:
blah = [1,2,3]

In [ ]:
push!(blah, 4)

In [ ]:
pop!(blah)

In [ ]:
blah

In [ ]:
function one_button_push_with_rx(states)
    not_found = true

    # A signal has three components: from, to, is_high
    queuing = [("button", "broadcaster", false)]

    while length(queuing) > 0 
        (from, to, is_high) = popfirst!(queuing)
        not_found = (to != "rx") || (is_high)
        # println((from, to, is_high))
        next_bool = handle_signal(from, to, is_high, states)
        if next_bool != nothing
            for n in states[to]["output"]
                push!(queuing, (to, n, next_bool))
            end
        end
        # println("Queuing is $queuing")
        # println(states)
    end
    # println(states)
    
    return not_found
end

In [ ]:
(p) = open("20_1.txt") do f 

    all_modules = DefaultDict(() -> DefaultDict(() -> Dict()))
    all_output_keys = Set()
    # all_modules["output"] = Dict("type" => "output", "input" => [], "states" => Dict(), "output" => [], "is_off" => true)
    
    for l in eachline(f)
        (emitter, receiver) = split(l, " -> ")
        mod_name = emitter[2:end]
        type_name = emitter[1:1]
        if emitter == "broadcaster"
            mod_name = emitter
            type_name = emitter
        end
        for o in split(receiver, ", ")
            push!(all_output_keys, o)
        end
        
        all_modules[mod_name] = Dict("type" => type_name, "input" => [], "states" => Dict(), "output" => split(receiver, ", "), "is_off" => true)
            
    end

    for o in all_output_keys
        if !(o in keys(all_modules ))
            all_modules[o] = Dict("type" => "void", "input" => [], "states" => Dict(), "output" => [], "is_off" => true)
        end
    end
    
    
    println(all_modules)

    for mod_name in keys(all_modules)
        for a_receiver in all_modules[mod_name]["output"]
            println("Current mod name is $mod_name and receiver is $a_receiver")
            push!(all_modules[a_receiver]["input"], mod_name)
            all_modules[a_receiver]["states"][mod_name] = false # false means low pulse in memory
        end
    end

    println(all_modules)

    keep_going = true
    nb_press = 0

    while keep_going
        keep_going = one_button_push_with_rx(all_modules)
        nb_press += 1
    end

    
    (nb_press)
end

In [38]:
function one_button_push_with_look_up(states, target_module, target_value)
    # not_found = true
    println("Pushing the button!")

    # A signal has three components: from, to, is_high
    queuing = [("button", "broadcaster", false)]

    while length(queuing) > 0 
        (from, to, is_high) = popfirst!(queuing)
        if (to == target_module) && (is_high == target_value )
            println((from, to, is_high))
            return false
        end
        # println((from, to, is_high))
        next_bool = handle_signal(from, to, is_high, states)
        if next_bool != nothing
            for n in states[to]["output"]
                push!(queuing, (to, n, next_bool))
            end
        end
        # println("Queuing is $queuing")
        # println(states)
    end
    # println(states)
    
    return true
end

one_button_push_with_look_up (generic function with 1 method)

In [39]:
(p) = open("20_1.txt") do f 

    all_modules = DefaultDict(() -> DefaultDict(() -> Dict()))
    all_output_keys = Set()
    # all_modules["output"] = Dict("type" => "output", "input" => [], "states" => Dict(), "output" => [], "is_off" => true)
    
    for l in eachline(f)
        (emitter, receiver) = split(l, " -> ")
        mod_name = emitter[2:end]
        type_name = emitter[1:1]
        if emitter == "broadcaster"
            mod_name = emitter
            type_name = emitter
        end
        for o in split(receiver, ", ")
            push!(all_output_keys, o)
        end
        
        all_modules[mod_name] = Dict("type" => type_name, "input" => [], "states" => Dict(), "output" => split(receiver, ", "), "is_off" => true)
            
    end

    for o in all_output_keys
        if !(o in keys(all_modules ))
            all_modules[o] = Dict("type" => "void", "input" => [], "states" => Dict(), "output" => [], "is_off" => true)
        end
    end

    for mod_name in keys(all_modules)
        for a_receiver in all_modules[mod_name]["output"]
            println("Current mod name is $mod_name and receiver is $a_receiver")
            push!(all_modules[a_receiver]["input"], mod_name)
            all_modules[a_receiver]["states"][mod_name] = false # false means low pulse in memory
        end
    end


    input_to_rx = all_modules[all_modules["rx"]["input"][1]]["input"]
    min_pushes = Vector{Int}(undef, length(input_to_rx))
    for (idx,a_module) in enumerate(input_to_rx)
        println("Currently looking for $a_module")
        new_modules = deepcopy(all_modules)
        nb_pushes = 0
        keep_going = true
        while keep_going
            nb_pushes += 1
            if (nb_pushes % 10000) == 0
                println("nb currently is $nb_pushes")
            end
            keep_going = one_button_push_with_look_up(new_modules, a_module, false)
            println("In the main loop, keep going is $keep_going")
        end
        min_pushes[idx] = nb_pushes
    end
    
    keep_going = true
    nb_press = 0


    
    (lcm(min_pushes))
end

Current mod name is mh and receiver is jt
Current mod name is tl and receiver is zq
Current mod name is tl and receiver is hl
Current mod name is hb and receiver is mh
Current mod name is hb and receiver is mt
Current mod name is qc and receiver is cl
Current mod name is qc and receiver is zd
Current mod name is sp and receiver is xm
Current mod name is gd and receiver is gm
Current mod name is gd and receiver is mt
Current mod name is lf and receiver is sq
Current mod name is zb and receiver is zd
Current mod name is zb and receiver is ph
Current mod name is zn and receiver is kx
Current mod name is zn and receiver is tc
Current mod name is kg and receiver is lf
Current mod name is ld and receiver is zn
Current mod name is cg and receiver is mt
Current mod name is cg and receiver is hb
Current mod name is md and receiver is cg
Current mod name is vj and receiver is bz
Current mod name is vj and receiver is zd
Current mod name is qs and receiver is zd
Current mod name is qs and receive

Excessive output truncated after 524306 bytes.


Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the main loop, keep going is true
Pushing the button!
In the mai

241528477694627

In [30]:
(p) = open("20_test2.txt") do f 

    all_modules = DefaultDict(() -> DefaultDict(() -> Dict()))
    all_output_keys = Set()
    # all_modules["output"] = Dict("type" => "output", "input" => [], "states" => Dict(), "output" => [], "is_off" => true)
    
    for l in eachline(f)
        (emitter, receiver) = split(l, " -> ")
        mod_name = emitter[2:end]
        type_name = emitter[1:1]
        if emitter == "broadcaster"
            mod_name = emitter
            type_name = emitter
        end
        for o in split(receiver, ", ")
            push!(all_output_keys, o)
        end
        
        all_modules[mod_name] = Dict("type" => type_name, "input" => [], "states" => Dict(), "output" => split(receiver, ", "), "is_off" => true)
            
    end

    for o in all_output_keys
        if !(o in keys(all_modules ))
            all_modules[o] = Dict("type" => "void", "input" => [], "states" => Dict(), "output" => [], "is_off" => true)
        end
    end

    for mod_name in keys(all_modules)
        for a_receiver in all_modules[mod_name]["output"]
            println("Current mod name is $mod_name and receiver is $a_receiver")
            push!(all_modules[a_receiver]["input"], mod_name)
            all_modules[a_receiver]["states"][mod_name] = false # false means low pulse in memory
        end
    end


    input_to_rx = all_modules[all_modules["output"]["input"][1]]["input"]
    min_pushes = Vector{Int}(undef, length(input_to_rx))
    for (idx,a_module) in enumerate(input_to_rx)
        println("Currently looking for $a_module")
        new_modules = deepcopy(all_modules)
        nb_pushes = 0
        keep_going = true
        while keep_going
            nb_pushes += 1
            if (nb_pushes % 10000) == 0
                println("nb currently is $nb_pushes")
            end
            keep_going = one_button_push_with_look_up(new_modules, a_module, true)
            println("In the main loop, keep going is $keep_going")
        end
        min_pushes[idx] = nb_pushes
    end
    
    println(min_pushes)


    
    (lcm(min_pushes))
end

Current mod name is con and receiver is output
Current mod name is b and receiver is con
Current mod name is broadcaster and receiver is a
Current mod name is a and receiver is inv
Current mod name is a and receiver is con
Current mod name is inv and receiver is b
Currently looking for b
Pushing the button!
In the main loop, keep going is false
Currently looking for a
Pushing the button!
In the main loop, keep going is false
[1, 1]


1

In [29]:
Vector{Int}(undef, 3)

3-element Vector{Int64}:
 140708473271200
 140708391958816
 140708474806048